# PTT CVS Product Review Analysis

This project demonstrates a system for predicting product ratings based on reviews from PTT's CVS board using machine learning, specifically fine-tuning a pre-trained model. The tokenizer model used in this project is 'bert-base-chinese' and the pre-trained model is 'ckiplab/albert-tiny-chinese-ws', a variant of ALBERT for Traditional Chinese text.

In [2]:
pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install accelerate==0.26.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import transformers
from transformers import BertTokenizerFast, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
from sklearn.model_selection import train_test_split
import torch.backends
import warnings
from datasets import Dataset
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from torch.utils.tensorboard import SummaryWriter

2025-01-17 08:48:28.570898: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737103708.630358     979 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737103708.645585     979 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 08:48:28.791485: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
warnings.filterwarnings('ignore')


## Data Loading and Exploration
Load the data and explore its structure to ensure there are no errors or anomalies. Check the rating column's range, distribution, and other statistical details.

1. **Load the Data**: Import the CSV file and display the first few rows.
2. **Describe the Data**: Use `describe()` to get basic statistical information about the dataset.
3. **Check Rating Range**: Ensure that all ratings are within a valid range (0 to 100).
4. **View Rating Distribution**: Check the distribution of ratings and calculate the average rating for each store.

In [3]:
df = pd.read_csv("cvs_products.csv")

In [4]:
df.head()

,product,link,store,CVS,rating,review
0,辻利抹茶歐蕾 38,https://www.ptt.cc/bbs/CVS/M.1570449843.A.7FF....,全家,全家,30,難喝.. 喝起來沒有抹茶味 只有一股很噁的甜味和水味 為了確認又喝幾口 真的難喝 大家就避開...
1,香辣霸王雞球/原價39元（友善價28元）,https://www.ptt.cc/bbs/CVS/M.1570452664.A.B42....,全家,全家,60,基本上在便利商店買這個就是解嘴饞 （因為要吃的鹹酥雞都沒開...） 然後一如其它炸物 微波後...
2,黑松C&C氣泡飲(王林蘋果風味) / 原價$29 全家購入特價$22,https://www.ptt.cc/bbs/CVS/M.1570461438.A.947....,全家 / 黑松,全家,90,最近好多蘋果氣泡飲料喔 看到綠色瓶身很漂亮，又有嚐鮮價 立刻忍不住試試XD 和其他蘋果汽水比...
3,美少女戰士悠遊閃卡-戰士款/100元,https://www.ptt.cc/bbs/CVS/M.1570494716.A.C35....,7-11 / 悠遊卡股份有限公司,7-11,100,第一彈其實完全沒有follow到， 偶然在板上看到第二彈情報文，結果覺得更生火！ 身為兒時禮...
4,統一麵包紫薯QQ 35,https://www.ptt.cc/bbs/CVS/M.1570502732.A.406....,7-11 統一麵包,7-11,88,早上到小七挑早餐看到紫薯覺得很特別就買來吃吃看 麵包整個顏色偏紫 味道聞起來甜甜 有淡淡的紫...


In [5]:
df['rating'].unique()

array([ 30,  60,  90, 100,  88,  68,  95,  75,  85,  80,  70,  40,  98,
        86,  65,  79,  59,  39,  89,  55,  49,  66,  83,  92,  84,  58,
         3,  50,  87,  78,  15,  99,  61,  72,  20,  81,  82,  93,   0,
        73,  10,  56,   7,  96,  74,  67,   8,  63,  76,   1,  69,  45,
         5,  77,  33,  62,  48,  94,  19,  42,  35,  25,  29,  91,  23,
        11,  32,  64,  71,  43,  31,   2,  97,  28])

In [6]:
rating_counts = df['rating'].value_counts().sort_values(ascending=False)
print(rating_counts.head(20))

rating
80     1835
85     1459
90     1184
70      974
75      922
60      422
65      374
88      234
95      234
50      199
100     189
59      133
82      130
40      100
78       99
30       95
83       95
0        92
79       56
55       55
Name: count, dtype: int64


In [7]:
average_ratings = df.groupby('CVS')['rating'].mean()
print(average_ratings)

CVS
7-11    75.067762
OK      74.958042
全家      76.982945
萊爾富     76.654160
Name: rating, dtype: float64


In [8]:
df.describe()

,rating
count,9618.000000
mean,76.253067
std,16.159308
min,0.000000
25%,70.000000
50%,80.000000
75%,85.000000
max,100.000000


## Data Cleaning and Preprocessing
Clean the data by removing unnecessary columns and standardizing the ratings to make the data suitable for model training.

1. **Data Cleaning**: Remove columns that are not needed for the model (keep only `review` and `rating`).
2. **Standardization**: Standardize the rating column to ensure it is scaled appropriately for regression models.
3. **Train-Test Split**: Split the data into training (90%) and testing (10%) sets for model training and evaluation.

In [9]:
df1 = df.drop(['product', 'store', 'link', 'CVS'], axis = 1)

In [10]:
df1.head()

,rating,review
0,30,難喝.. 喝起來沒有抹茶味 只有一股很噁的甜味和水味 為了確認又喝幾口 真的難喝 大家就避開...
1,60,基本上在便利商店買這個就是解嘴饞 （因為要吃的鹹酥雞都沒開...） 然後一如其它炸物 微波後...
2,90,最近好多蘋果氣泡飲料喔 看到綠色瓶身很漂亮，又有嚐鮮價 立刻忍不住試試XD 和其他蘋果汽水比...
3,100,第一彈其實完全沒有follow到， 偶然在板上看到第二彈情報文，結果覺得更生火！ 身為兒時禮...
4,88,早上到小七挑早餐看到紫薯覺得很特別就買來吃吃看 麵包整個顏色偏紫 味道聞起來甜甜 有淡淡的紫...


In [11]:
scaler = StandardScaler()
df1['rating_standard'] = scaler.fit_transform(df['rating'].values.reshape(-1, 1))

In [12]:
df1.head()

,rating,review,rating_standard
0,30,難喝.. 喝起來沒有抹茶味 只有一股很噁的甜味和水味 為了確認又喝幾口 真的難喝 大家就避開...,-2.862466
1,60,基本上在便利商店買這個就是解嘴饞 （因為要吃的鹹酥雞都沒開...） 然後一如其它炸物 微波後...,-1.005854
2,90,最近好多蘋果氣泡飲料喔 看到綠色瓶身很漂亮，又有嚐鮮價 立刻忍不住試試XD 和其他蘋果汽水比...,0.850757
3,100,第一彈其實完全沒有follow到， 偶然在板上看到第二彈情報文，結果覺得更生火！ 身為兒時禮...,1.469628
4,88,早上到小七挑早餐看到紫薯覺得很特別就買來吃吃看 麵包整個顏色偏紫 味道聞起來甜甜 有淡淡的紫...,0.726983


In [13]:
X_train, X_test, y_train, y_test = train_test_split(df1['review'], df1['rating_standard'], test_size=0.1, random_state=42)

## Model Building and Training
Set up and initialize the model, and train it using the Hugging Face Transformers library.

1. **Model Selection and Initialization**: Use the pre-trained `bert-base-chinese` tokenizer and the `albert-tiny-chinese-ws` model to process the Chinese reviews.
2. **Model Structure Adjustment**: Since this is a regression task, we will modify the model's final layer to be a linear layer for regression.
3. **Training and Evaluation**: Set up training arguments, train the model, and evaluate its performance.

In [14]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForSequenceClassification.from_pretrained(
    'ckiplab/albert-tiny-chinese-ws',
    num_labels=1,
    ignore_mismatched_sizes=True
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ckiplab/albert-tiny-chinese-ws and are newly initialized: ['albert.pooler.bias', 'albert.pooler.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ckiplab/albert-tiny-chinese-ws and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 312]) in the checkpoint and torch.Size([1, 312]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.config

AlbertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "ckiplab/albert-tiny-chinese-ws",
  "architectures": [
    "AlbertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 101,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 102,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 1248,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 4,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizerFast",
  "transformers_version": "4.48.0",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

In [16]:
model.classifier = torch.nn.Linear(model.config.hidden_size, 1)

In [17]:
train_df = pd.DataFrame({'text': X_train, 'label': y_train.astype(float)})
test_df = pd.DataFrame({'text': X_test, 'label': y_test.astype(float)})

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [18]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

In [19]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

In [20]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8656 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

In [23]:
training_args = TrainingArguments(
    output_dir='./results/ckiplab-albert-tiny',
    num_train_epochs=10,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs/ckiplab-albert-tiny', 
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch", 
    load_best_model_at_end=True 
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [25]:
writer = SummaryWriter()

for epoch in range(training_args.num_train_epochs):
    trainer.train()

    predictions, labels, _ = trainer.predict(test_dataset)
    
    mse = mean_squared_error(labels, predictions)
    r2 = r2_score(labels, predictions)
    writer.add_scalar('MSE', mse, epoch)
    writer.add_scalar('R-squared', r2, epoch)

writer.close()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [45]:
trainer.train()

  0%|          | 0/5410 [00:00<?, ?it/s]

{'loss': 642.2259, 'grad_norm': 1558.442138671875, 'learning_rate': 1e-05, 'epoch': 0.18}
{'loss': 2.5234, 'grad_norm': 12.049978256225586, 'learning_rate': 2e-05, 'epoch': 0.37}
{'loss': 1.2878, 'grad_norm': 31.094709396362305, 'learning_rate': 3e-05, 'epoch': 0.55}
{'loss': 1.2702, 'grad_norm': 283.6398620605469, 'learning_rate': 4e-05, 'epoch': 0.74}
{'loss': 1.1096, 'grad_norm': 21.44092559814453, 'learning_rate': 5e-05, 'epoch': 0.92}


  0%|          | 0/61 [00:00<?, ?it/s]

{'eval_loss': 1.3721643686294556, 'eval_runtime': 10.384, 'eval_samples_per_second': 92.642, 'eval_steps_per_second': 5.874, 'epoch': 1.0}
{'loss': 1.0478, 'grad_norm': 26.937990188598633, 'learning_rate': 4.89816700610998e-05, 'epoch': 1.11}
{'loss': 1.0277, 'grad_norm': 148.75587463378906, 'learning_rate': 4.79633401221996e-05, 'epoch': 1.29}
{'loss': 1.0407, 'grad_norm': 89.99839782714844, 'learning_rate': 4.694501018329939e-05, 'epoch': 1.48}
{'loss': 1.0624, 'grad_norm': 87.89129638671875, 'learning_rate': 4.592668024439919e-05, 'epoch': 1.66}
{'loss': 1.082, 'grad_norm': 75.37065124511719, 'learning_rate': 4.490835030549899e-05, 'epoch': 1.85}


  0%|          | 0/61 [00:00<?, ?it/s]

...

  0%|          | 0/61 [00:00<?, ?it/s]

{'eval_loss': 0.7342021465301514, 'eval_runtime': 10.8341, 'eval_samples_per_second': 88.794, 'eval_steps_per_second': 5.63, 'epoch': 9.0}
{'loss': 0.0426, 'grad_norm': 5.722010135650635, 'learning_rate': 5.193482688391039e-06, 'epoch': 9.06}
{'loss': 0.0262, 'grad_norm': 5.641330242156982, 'learning_rate': 4.175152749490835e-06, 'epoch': 9.24}
{'loss': 0.0356, 'grad_norm': 5.461765766143799, 'learning_rate': 3.1568228105906318e-06, 'epoch': 9.43}
{'loss': 0.0279, 'grad_norm': 4.584555625915527, 'learning_rate': 2.1384928716904276e-06, 'epoch': 9.61}
{'loss': 0.0244, 'grad_norm': 8.97751235961914, 'learning_rate': 1.120162932790224e-06, 'epoch': 9.8}
{'loss': 0.0246, 'grad_norm': 5.923080921173096, 'learning_rate': 1.0183299389002036e-07, 'epoch': 9.98}


  0%|          | 0/61 [00:00<?, ?it/s]

{'eval_loss': 0.7300607562065125, 'eval_runtime': 10.3447, 'eval_samples_per_second': 92.994, 'eval_steps_per_second': 5.897, 'epoch': 10.0}
{'train_runtime': 2568.029, 'train_samples_per_second': 33.707, 'train_steps_per_second': 2.107, 'train_loss': 12.326706225296926, 'epoch': 10.0}


TrainOutput(global_step=5410, training_loss=12.326706225296926, metrics={'train_runtime': 2568.029, 'train_samples_per_second': 33.707, 'train_steps_per_second': 2.107, 'total_flos': 348519843594240.0, 'train_loss': 12.326706225296926, 'epoch': 10.0})

In [46]:
eval_results = trainer.evaluate()

print(f"Validation Loss: {eval_results['eval_loss']}")

  0%|          | 0/61 [00:00<?, ?it/s]

Validation Loss: 0.688459038734436


In [47]:
predictions, labels, _ = trainer.predict(test_dataset)

  0%|          | 0/61 [00:00<?, ?it/s]

In [48]:

mse = mean_squared_error(labels, predictions)